In [1]:
import numpy as np
import os
import networkx as nx
from scipy.ndimage import zoom
def load_and_validate_data(file_path):
    """
    Load data from a file and return it.
    Return None if data is missing.
    """
    try:
        loaded_data = np.load(file_path)
        
        ppg_f = loaded_data.get('ppg_f')
        ecg_f = loaded_data.get('ecg_f')
        seg_dbp = loaded_data.get('seg_dbp')
        seg_sbp = loaded_data.get('seg_sbp')
        
        
        if ppg_f is None or ecg_f is None or seg_dbp is None or seg_sbp is None:
            return None

        return ppg_f, ecg_f, seg_dbp, seg_sbp
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def combine_data_from_folder(folder_path, batch_size=100):
    """
    Combine data from all valid files in the folder in batches.
    """
    combined_ppg = []
    combined_ecg = []
    combined_seg_dbp = []
    combined_seg_sbp = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not file_path.endswith('.npz'):
            continue
        
        data = load_and_validate_data(file_path)
        
        if data is None:
            print(f"Skipping invalid file: {file_path}")
            continue
        
        ppg_f, ecg_f, seg_dbp, seg_sbp = data
        
        combined_ppg.append(ppg_f)
        combined_ecg.append(ecg_f)
        combined_seg_dbp.append(seg_dbp)
        combined_seg_sbp.append(seg_sbp)
        
        if len(combined_ppg) >= batch_size:
            combined_ppg = np.concatenate(combined_ppg, axis=0)
            combined_ecg = np.concatenate(combined_ecg, axis=0)
            combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
            combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
            
            yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
            
            combined_ppg = []
            combined_ecg = []
            combined_seg_dbp = []
            combined_seg_sbp = []
            

    if combined_ppg:
        combined_ppg = np.concatenate(combined_ppg, axis=0)
    else:
        combined_ppg = np.array([])
        
    if combined_ecg:
        combined_ecg = np.concatenate(combined_ecg, axis=0)
    else:
        combined_ecg = np.array([])
        
    if combined_seg_dbp:
        combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
    else:
        combined_seg_dbp = np.array([])
        
    if combined_seg_sbp:
        combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
    else:
        combined_seg_sbp = np.array([])

    yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
train_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Train_data'
val_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Validation_data'
test_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Test_data'

In [2]:
def create_visibility_graph(ppg_signal):
    n = len(ppg_signal)
    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in range(n):
        for j in range(i + 1, n):
            visible = True
            for k in range(i + 1, j):
                if ppg_signal[k] >= ppg_signal[i] + (ppg_signal[j] - ppg_signal[i]) * (k - i) / (j - i):
                    visible = False
                    break
            if visible:
                G.add_edge(i, j)
                
    return G

def graph_to_adjacency_matrix_image(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    return adj_matrix_resized

def graph_to_flattened_adjacency_matrix(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    flattened_adj = adj_matrix_resized.flatten()
    return flattened_adj[:size * size]  

def generate_vg_image(ppg_signal, size):
    G = create_visibility_graph(ppg_signal)
    vg_image = graph_to_adjacency_matrix_image(G, size)
    return vg_image

def process_signal(i, ppg_signal, vg_image_size):
    """
    Generate a VG image for a given PPG signal.
    """
    #print(f"VG img {i + 1}")
    ppg_signal = ppg_signal.flatten()
    vg_image = generate_vg_image(ppg_signal, vg_image_size)
    return vg_image.flatten()

vg_image_size=224

In [3]:
import numpy as np
from scipy.signal import find_peaks

def select_n_peak_window(ppg_signal, n_peaks=3):
    ppg_3peak=[]
    for ppg in ppg_signal:
        x=ppg.flatten()
        peaks, _ = find_peaks(x, distance=50)  
    
    
    
        if len(peaks) < n_peaks:
            raise ValueError(f"Not enough peaks detected. Detected peaks: {len(peaks)}")
    
    
        start_index = peaks[0]
        end_index = peaks[n_peaks - 1]  
    
    
        ppg_window = x[start_index:end_index + 1]
        ppg_3peak.append(np.array(ppg_window))
    
    return ppg_3peak

def reshape_ppg_3_peaks(ppg_signal):
    ppg_3_peak_reshaped = []

    for ppg_window in ppg_signal:
        reshaped_window = ppg_window.reshape(1, -1)  
        ppg_3_peak_reshaped.append(np.array(reshaped_window))
        
    return ppg_3_peak_reshaped

def reshape_ppg_3_peaks_224(ppg_signal):
    ppg_3_peak_reshaped = []

    for ppg_window in ppg_signal:
        window = ppg_window[0:224]
        reshaped_window = ppg_window.reshape(1, -1)  
        ppg_3_peak_reshaped.append(np.array(reshaped_window))
        
    return ppg_3_peak_reshaped

In [4]:
val_data_generator = combine_data_from_folder(val_dir, batch_size=2)

In [5]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import os

vg_image_dir = 'vg_images_validation'


def load_vg_images(batch_idx):
    vg_images_path = os.path.join(vg_image_dir, f'val_vg_images_batch_{batch_idx}.npz')
    
    if not os.path.exists(vg_images_path) :
        print(f"VG image files do not exist for Batch {batch_idx}.")
        return None
    
    vg_images = np.load(vg_images_path)['vg_images']

    
    return vg_images

def plot_vg_images(vg_images, num_images=5, image_size=(224, 224)):
    for i in range(min(num_images, vg_images.shape[0])):
        plt.figure(figsize=(5, 5))
        # Reshape each VG image to its original 2D shape
        img = vg_images[i].reshape(image_size)
        plt.imshow(img)
        plt.title(f'VG Image {i+1}')
        plt.axis('off')
        plt.show()


# With Val data

In [6]:

all_vg_images = []
all_sbp_values = []
all_dbp_values = []
#all_ppg_node_values = []

for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(val_data_generator):
    print(f"Processing Batch {batch_idx + 1}...")
    
    #if batch_idx + 1>2:
       # break
    vg_images = load_vg_images(batch_idx + 1)
    if vg_images is None:
        print(f"No VG images found for Batch {batch_idx + 1}. Stopping...")
        break
    
    flattened_vg_images = np.array(vg_images)
    #ppg_peaks_3=select_n_peak_window(combined_ppg_batch)
    #ppg_peaks_3_reshaped=reshape_ppg_3_peaks(ppg_peaks_3)
    #ppg_peaks_3_reshaped_224=reshape_ppg_3_peaks(combined_ppg_batch)
    #all_ppg_node_values.append(np.array(ppg_peaks_3_reshaped_224))
    all_vg_images.append(flattened_vg_images)
    all_sbp_values.append(np.array(combined_seg_sbp_batch.flatten()))
    all_dbp_values.append(np.array(combined_seg_dbp_batch.flatten()))

    print(f"Batch {batch_idx + 1} processing complete.")
    
#all_ppg_node_values = np.concatenate(all_ppg_node_values, axis=0)
all_vg_images = np.concatenate(all_vg_images, axis=0)
all_sbp_values = np.concatenate(all_sbp_values, axis=0)
all_dbp_values = np.concatenate(all_dbp_values, axis=0)     

Processing Batch 1...
Batch 1 processing complete.
Processing Batch 2...
Batch 2 processing complete.
Processing Batch 3...
Batch 3 processing complete.
Processing Batch 4...
Batch 4 processing complete.
Processing Batch 5...
VG image files do not exist for Batch 5.
No VG images found for Batch 5. Stopping...


In [7]:
y = np.column_stack((all_sbp_values, all_dbp_values))

In [8]:
print(len(all_vg_images))
print(len(all_sbp_values))

4149
4149


In [8]:
import numpy as np
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.linear_model import Ridge


vg_images_normal = np.array(all_vg_images)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical

# Define the batch size
batch_size = 32

# Function to generate image batches
def image_generator(images, labels, batch_size):
    num_samples = images.shape[0]
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_images = images[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]
            batch_images_reshaped = batch_images.reshape(-1, 224, 224, 1)
            batch_images_rgb = np.concatenate([batch_images_reshaped] * 3, axis=-1)
            batch_images_preprocessed = preprocess_input(batch_images_rgb)
            yield batch_images_preprocessed, batch_labels


In [10]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = Flatten(name='flatten')(x)
#x = Dense(4096, activation='relu', name='fc1')(x)
#x = Dense(4096, activation='relu', name='fc2')(x)
output = Dense(512, activation='linear',name='fc1')(x)  


model = Model(inputs=base_model.input, outputs=output)


model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
images = vg_images_normal
labels = y
num_samples = images.shape[0]

In [12]:
# Define a new model to extract features from the 'fc1' layer of the trained model
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('fc1').output)

# Function to extract features
def extract_features(generator, num_samples):
    features = []
    for batch_images, _ in generator:
        batch_features = feature_extractor.predict(batch_images)
        features.append(batch_features)
        if len(features) * batch_size >= num_samples:
            break
    return np.vstack(features)


feature_generator_normal = image_generator(images, np.zeros((num_samples,)), batch_size)


features_normal = extract_features(feature_generator_normal, num_samples)


features_normal_flattened= features_normal.reshape(features_normal.shape[0],-1)


features= features_normal_flattened

1/1 [==============================] - 3s 3s/step


In [13]:
from sklearn.model_selection import train_test_split

vg_features_train,vg_features_test,label_vgg_train,label_vgg_test=train_test_split(features,labels, test_size=0.2, random_state=42)
ridge_model = Ridge(alpha=1.0)  
ridge_model.fit(vg_features_train, label_vgg_train)


predicted_values_split = ridge_model.predict(vg_features_test)

In [14]:
predicted_sbp =predicted_values_split[:, 0]  # Assuming the first column is SBP
predicted_dbp =predicted_values_split[:, 1]  # Assuming the second column is DBP

true_sbp = label_vgg_test[:, 0]  # True SBP values from q
true_dbp = label_vgg_test[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

SBP Metrics: R=0.5744753672137302, MAE=11.443205502763947, RMSE=15.437279804635498, ME±SD=(0.30231606200353617, 15.434319316551074)
DBP Metrics: R=0.6478334648846578, MAE=6.440628848545877, RMSE=8.834369097467382, ME±SD=(0.060790584063006914, 8.834159940547599)


In [15]:
num_samples

10702

In [19]:
all_vg_images[0]

array([0., 1., 1., ..., 1., 1., 0.])

In [24]:
for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(train_data_generator):
    peak=select_n_peak_window(combined_ppg_batch)
    peak_reshaped=reshape_ppg_3_peaks(peak)
    break

In [27]:
peak_reshaped[0]

array([[3.62155461e-01, 3.61261763e-01, 3.57333689e-01, 3.50510351e-01,
        3.41094774e-01, 3.29426638e-01, 3.15972071e-01, 3.01228797e-01,
        2.85693173e-01, 2.69866566e-01, 2.54180109e-01, 2.39038403e-01,
        2.24749350e-01, 2.11488633e-01, 1.99420712e-01, 1.88569369e-01,
        1.78900548e-01, 1.70347126e-01, 1.62707639e-01, 1.55884837e-01,
        1.49698104e-01, 1.43946993e-01, 1.38564384e-01, 1.33369253e-01,
        1.28320207e-01, 1.23440963e-01, 1.18663800e-01, 1.14022183e-01,
        1.09640159e-01, 1.05578577e-01, 1.01859327e-01, 9.86060221e-02,
        9.59081935e-02, 9.37782857e-02, 9.22286129e-02, 9.12446371e-02,
        9.08007807e-02, 9.08296620e-02, 9.11501782e-02, 9.16466019e-02,
        9.22481286e-02, 9.27061253e-02, 9.28995333e-02, 9.27902529e-02,
        9.22410298e-02, 9.12210127e-02, 8.97845497e-02, 8.80049087e-02,
        8.59567242e-02, 8.37907948e-02, 8.16973394e-02, 7.98209804e-02,
        7.83189467e-02, 7.73265749e-02, 7.69107144e-02, 7.709496

In [28]:
vg_images_normal

array([[0., 1., 1., ..., 1., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.]])

In [29]:
all_vg_images

array([[0., 1., 1., ..., 1., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.]])

# MobileNet

In [9]:
vg_images_normal = np.array(all_vg_images)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.linear_model import Ridge

# Define the batch size
batch_size = 32

# Function to generate image batches
def image_generator(images, labels, batch_size):
    num_samples = images.shape[0]
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_images = images[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]
            batch_images_reshaped = batch_images.reshape(-1, 224, 224, 1)
            batch_images_rgb = np.concatenate([batch_images_reshaped] * 3, axis=-1)
            batch_images_preprocessed = preprocess_input(batch_images_rgb)
            yield batch_images_preprocessed, batch_labels


In [11]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet import preprocess_input


base_model_mobile = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


for layer in base_model_mobile.layers:
    layer.trainable = False


x = base_model_mobile.output
x = Flatten(name='flatten')(x)
output = Dense(512, activation='linear', name='fc1')(x)


model_mobile = Model(inputs=base_model_mobile.input, outputs=output)

model_mobile.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128   

In [12]:
images = vg_images_normal
labels = y
num_samples = images.shape[0]

In [13]:
# Define a new model to extract features from the 'fc1' layer of the trained model
feature_extractor_mobile = Model(inputs=model_mobile.input, outputs=model_mobile.get_layer('fc1').output)

# Function to extract features
def extract_features(generator, num_samples):
    features = []
    for batch_images, _ in generator:
        batch_features = feature_extractor_mobile.predict(batch_images)
        features.append(batch_features)
        if len(features) * batch_size >= num_samples:
            break
    return np.vstack(features)


feature_generator_normal_1 = image_generator(images, np.zeros((num_samples,)), batch_size)

features_normal_1 = extract_features(feature_generator_normal_1, num_samples)

features_normal_flattened_1= features_normal_1.reshape(features_normal_1.shape[0],-1)

features_1= features_normal_flattened_1

1/1 [==============================] - 1s 737ms/step


In [14]:
from sklearn.model_selection import train_test_split

Mobile_train,Mobile_test,label_train,label_test=train_test_split(features_1,labels, test_size=0.2, random_state=42)
ridge_model = Ridge(alpha=1.0)  
ridge_model.fit(Mobile_train, label_train)


predicted_mobile = ridge_model.predict(Mobile_test)

In [15]:
predicted_sbp =predicted_mobile[:, 0]  # Assuming the first column is SBP
predicted_dbp =predicted_mobile[:, 1]  # Assuming the second column is DBP

true_sbp = label_test[:, 0]  # True SBP values from q
true_dbp = label_test[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

SBP Metrics: R=0.7496319155604708, MAE=9.518621532486849, RMSE=12.385240064082302, ME±SD=(-0.5771832976955933, 12.371783658220451)
DBP Metrics: R=0.6958436174050171, MAE=3.8236322081578247, RMSE=5.347403835193292, ME±SD=(-0.07850590920669992, 5.346827526567465)


# FULL DATASET

In [6]:
import numpy as np
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from sklearn.linear_model import Ridge

# Define VGG19 model for feature extraction
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten(name='flatten')(x)
output = Dense(512, activation='linear', name='fc1')(x)
feature_extractor = Model(inputs=base_model.input, outputs=output)

# Initialize Ridge Regression model
from sklearn.linear_model import SGDRegressor

# Initialize SGD Regressor model
sbp_model = SGDRegressor()
dbp_model = SGDRegressor()


def process_and_train(batch_idx, vg_images, sbp_values, dbp_values):
    # Reshape VG images for VGG19 input
    print(f"Batch {batch_idx + 1} processing for training.")
    vg_images_reshaped = vg_images.reshape(-1, 224, 224, 1)
    vg_images_rgb = np.concatenate([vg_images_reshaped] * 3, axis=-1)
    
    # Preprocess images
    vg_images_preprocessed = preprocess_input(vg_images_rgb)
    
    # Extract features using VGG19
    #if tf.config.list_physical_devices('GPU'):
        #with tf.device('/GPU:0'):
            #features = feature_extractor.predict(vg_images_preprocessed, batch_size=32)
    #else:
        #features = feature_extractor.predict(vg_images_preprocessed, batch_size=32)
        
    features = feature_extractor.predict(vg_images_preprocessed)
    
    # Flatten features
    features_flattened = features.reshape(features.shape[0], -1)
    
    # Combine SBP and DBP into a single array
    #labels = np.column_stack((sbp_values, dbp_values))
    
    # Train Ridge Regression model
    sbp_model.partial_fit(features_flattened, sbp_values)
    dbp_model.partial_fit(features_flattened, dbp_values)
    print(f"Batch {batch_idx + 1} processed and used for training.")

In [ ]:
# Iterate over batches
for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(train_data_generator):
    # Load corresponding VG images
    vg_images = load_vg_images(batch_idx + 1)
    #print(np.array(vg_images))
    vg_images=np.array(vg_images)
    if vg_images is None:
        print(f"No VG images found for Batch {batch_idx + 1}. Stopping...")
        break
    
    # Process and train on the current batch
    process_and_train(batch_idx, vg_images, combined_seg_sbp_batch, combined_seg_dbp_batch)

print("Training complete.")

Batch 1 processing for training.
159/159 [==============================] - 795s 5s/step


C:\Users\nihal\anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\nihal\anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Batch 1 processed and used for training.
Batch 2 processing for training.
 92/177 [==============>...............] - ETA: 7:45

In [ ]:
test_data_generator = combine_data_from_folder(test_dir, batch_size=10)

In [ ]:
import numpy as np
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


def process_and_predict(batch_idx, vg_images, sbp_values, dbp_values):
    """
    Process VG images and predict SBP and DBP using Ridge Regression.
    """
    print(f"Batch {batch_idx + 1} processing for predicting.")
    
    # Reshape VG images for VGG19 input
    vg_images_reshaped = vg_images.reshape(-1, 224, 224, 1)
    vg_images_rgb = np.concatenate([vg_images_reshaped] * 3, axis=-1)
    
    # Preprocess images
    vg_images_preprocessed = preprocess_input(vg_images_rgb)
    
    # Extract features using VGG19
    features = feature_extractor.predict(vg_images_preprocessed)
    
    # Flatten features
    features_flattened = features.reshape(features.shape[0], -1)
    
    # Combine SBP and DBP into a single array
    labels = np.column_stack((sbp_values, dbp_values))
    
    # Predict using Ridge Regression model
    sbp_predictions = sbp_model.predict(features_flattened)
    dbp_predictions = dbp_model.predict(features_flattened)
    predictions = np.column_stack((sbp_predictions, dbp_predictions))
    
    # Calculate metrics for the current batch
    mae = mean_absolute_error(labels, predictions)
    rmse = np.sqrt(mean_squared_error(labels, predictions))
    me = np.mean(predictions - labels, axis=0)
    sd = np.std(predictions - labels, axis=0)
    r2 = r2_score(labels, predictions)
    
    print(f"Batch {batch_idx + 1} - MAE: {mae:.4f}, RMSE: {rmse:.4f}, ME: {me}, SD: {sd}, R2: {r2:.4f}")
    
    return mae, rmse, me, sd, r2

In [ ]:
def evaluate_metrics_in_batches(train_data_generator):
    batch_mae = []
    batch_rmse = []
    batch_me = []
    batch_sd = []
    batch_r2 = []

    for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(train_data_generator):
        # Load corresponding VG images
        vg_images = load_vg_images(batch_idx + 1)
        if vg_images is None:
            print(f"No VG images found for Batch {batch_idx + 1}. Stopping...")
            break
        
        # Process and predict on the current batch
        mae, rmse, me, sd, r2 = process_and_predict(batch_idx, vg_images, combined_seg_sbp_batch, combined_seg_dbp_batch)
        
        # Append metrics to lists
        batch_mae.append(mae)
        batch_rmse.append(rmse)
        batch_me.append(me)
        batch_sd.append(sd)
        batch_r2.append(r2)
    
    # Compute overall metrics
    overall_mae = np.mean(batch_mae)
    overall_rmse = np.mean(batch_rmse)
    overall_me = np.mean(batch_me, axis=0)
    overall_sd = np.mean(batch_sd, axis=0)
    overall_r2 = np.mean(batch_r2)
    
    return overall_mae, overall_rmse, overall_me, overall_sd, overall_r2

In [ ]:
# Usage
overall_mae, overall_rmse, overall_me, overall_sd, overall_r2 = evaluate_metrics_in_batches(train_data_generator)
print(f"Overall MAE: {overall_mae:.4f}")
print(f"Overall RMSE: {overall_rmse:.4f}")
print(f"Overall ME: {overall_me}")
print(f"Overall SD: {overall_sd}")
print(f"Overall R2: {overall_r2:.4f}")